# Library

In [1]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import re
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

### chrome driver 실행

In [5]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

## 페이지 접속

In [28]:
# 페이지 주소
url = 'https://shopping.naver.com/ns/home'

### requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [29]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

### selenium 작동

In [ ]:
driver.get(url)
driver.implicitly_wait(random.randrange(2, 5))
# driver.maximize_window() # 브라우져 창 최대화

### 검색 창에 검색어 입력

In [38]:
search_keyword = '아이라인'

before_search_window_xpath = '//*[@id="gnb-gnb"]/div/div/div[2]/div[2]/button' # 전단계 검색 입력창
before_search_window = driver.find_element(By.XPATH, before_search_window_xpath)
# before_search_window = driver.find_element(By.CSS_SELECTOR, 'button._shoppingHomeSearch_button_gXyNO N=a:gnb.allsearch')
before_search_window.click()
time.sleep(random.randrange(1, 3))

In [39]:
search_window_xpath = '//*[@id="input_text"]' # 검색 입력창

search_window = driver.find_element(By.XPATH, search_window_xpath)
search_window.send_keys(search_keyword)
time.sleep(random.randrange(1, 3))
search_window.send_keys(Keys.RETURN)

In [40]:
# 스크롤해야 정보가 로드되는 페이지

# 페이지 스크롤 다운
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(random.uniform(1, 3))

## 입력 키워드 정보 수집 

In [54]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'아이라인 : 네이버+ 스토어 검색'

In [55]:
# 상품 섹션
products_section = soup.select_one('div#composite-card-list')
len(products_section)

2

In [56]:
# 상품 리스트
product_list = products_section.select('li.compositeCardContainer_composite_card_container__jr8cb div.basicProductCard_basic_product_card__TdrHT')
len(product_list)

150

## 🔧 이미지 링크
- 90번대 이미지 링크 오류 수정 필요

In [44]:
# 상품 썸네일 이미지 링크
order_number = 0
product_list[order_number].select('div.productCardThumbnail_thumbnail__KzO1N img')

[<img alt="" class="autoFitImg_auto_fit_img__fIpj4 autoFitImg_full_height__QCTGq productCardThumbnail_image__Li6iz scale" data-src="https://searchad-phinf.pstatic.net/MjAyMjExMjFfMjYy/MDAxNjY4OTk0NjAwMjEz.4rb_wGCPEtzNpJ1bzlKJt58W43NSPXTwVZnw6m_Ndzcg.ujl-TTtPnMxDoCHJCCPjyFjinsixF_X8JIpm1Zz7INQg.JPEG/1281382-05a25423-3bef-4eb2-add4-7bff101fa375.jpg?type=f300" src="https://searchad-phinf.pstatic.net/MjAyMjExMjFfMjYy/MDAxNjY4OTk0NjAwMjEz.4rb_wGCPEtzNpJ1bzlKJt58W43NSPXTwVZnw6m_Ndzcg.ujl-TTtPnMxDoCHJCCPjyFjinsixF_X8JIpm1Zz7INQg.JPEG/1281382-05a25423-3bef-4eb2-add4-7bff101fa375.jpg?type=f300" style="opacity: 1;"/>]

In [45]:
product_thumnail_img_src = product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4')['src']
# product_thumnail_img_src = product_list[order_number].find('img', class_='autoFitImg_auto_fit_img__fIpj4 autoFitImg_full_height__QCTGq productCardThumbnail_image__Li6iz scale').attrs['src']
product_thumnail_img_src

'https://searchad-phinf.pstatic.net/MjAyMjExMjFfMjYy/MDAxNjY4OTk0NjAwMjEz.4rb_wGCPEtzNpJ1bzlKJt58W43NSPXTwVZnw6m_Ndzcg.ujl-TTtPnMxDoCHJCCPjyFjinsixF_X8JIpm1Zz7INQg.JPEG/1281382-05a25423-3bef-4eb2-add4-7bff101fa375.jpg?type=f300'

In [46]:
# 상품 정보 섹션
order_number = 0
# 상품 제목
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc strong.basicProductCardInformation_title__Bc_Ng').text)

# 상품 가격

print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_original_price__jyZRY').text)
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_discount__F_ZXz').text)
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_inner_price__TctbK').text)

1+1 미크릿 워셔블 비건 슈퍼픽서 아이라이너
할인 전 판매가24,000원
31%할인
16,500원


In [47]:
# 상품 정보 섹션
order_number = 1
# 상품 제목
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc strong.basicProductCardInformation_title__Bc_Ng').text)

# 상품 가격

print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_original_price__jyZRY').text)
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_discount__F_ZXz').text)
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_inner_price__TctbK').text)

메리쏘드 릴타투 브러쉬 펜라이너 붓펜 아이라인
할인 전 판매가11,000원
5%할인
10,400원


In [48]:
# 상품 정보 섹션
    # 할인 하지 않는 상품 존재 : 7, 12, 24 등등
order_number = 23
# 상품 제목
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc strong.basicProductCardInformation_title__Bc_Ng').text)

# 상품 가격
try:
    print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_original_price__jyZRY').text)
except:
    print('할인없음')
try:
    print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_discount__F_ZXz').text)
except:
    print('할인없음')
print(product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_inner_price__TctbK').text)

해외마죠리카 마죠르카 라인 익스팬더 아이라이너 BK999 블랙
할인없음
할인없음
11,240원


# 🔧 수집한 데이터 전처리
- 원래가격, 할인률, 현재가격 숫자만 표기하도록 정제
- 원래가격, 할인률 nan 값 처리된 부분 확인

In [49]:
num_list, img_src_list, title_list, original_price_list, discount_rate_list, price_list = [], [], [], [], [], []

for i in range(len(product_list)):
    try:
        num = i + 1
        img_src = product_list[i].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4')['src']
        
        title = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc strong.basicProductCardInformation_title__Bc_Ng').text
        
        try:
            original_price = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_original_price__jyZRY').text
        except:
            original_price = np.nan
        try:
            discount_rate = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_discount__F_ZXz').text
        except:
            discount_rate = np.nan
        price = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc span.priceTag_inner_price__TctbK').text

        num_list.append(num)
        img_src_list.append(img_src)
        title_list.append(title)
        original_price_list.append(original_price)
        discount_rate_list.append(discount_rate)
        price_list.append(price)
    except:
        print(i)

In [50]:
len(num_list), len(img_src_list), len(title_list), len(original_price_list), len(discount_rate_list), len(price_list)

(100, 100, 100, 100, 100, 100)

In [51]:
data_dict = {
    '번호':num_list, 
    '이미지링크':img_src_list,
    '제품제목':title_list,
    '원래가격':original_price_list,
    '할인률':discount_rate_list,
    '현재가격':price_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,이미지링크,제품제목,원래가격,할인률,현재가격
0,1,https://searchad-phinf.pstatic.net/MjAyMjExMjF...,1+1 미크릿 워셔블 비건 슈퍼픽서 아이라이너,"할인 전 판매가24,000원",31%할인,"16,500원"
1,2,https://searchad-phinf.pstatic.net/MjAyMzA1MDh...,메리쏘드 릴타투 브러쉬 펜라이너 붓펜 아이라인,"할인 전 판매가11,000원",5%할인,"10,400원"
2,3,https://shopping-phinf.pstatic.net/main_825922...,[공식]메이블린 라인 타투 아이 라이너 크레용,"할인 전 판매가13,000원",25%할인,"9,750원"
3,4,https://shopping-phinf.pstatic.net/main_830583...,번지지않는 붓펜 아이라이너 점막 안번지는 저자극 그리기쉬운 아이라인 펜슬 붓 트임,"할인 전 판매가12,900원",23%할인,"9,900원"
4,5,https://shopping-phinf.pstatic.net/main_830583...,번지지않는 붓펜 아이라이너 점막 안번지는 저자극 그리기쉬운 아이라인 펜슬 붓 트임,"할인 전 판매가12,900원",23%할인,"9,900원"


In [52]:
df.tail()

,번호,이미지링크,제품제목,원래가격,할인률,현재가격
95,96,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",해외디올 아이라인 펜슬 온 스테이지 On Stage Crayon,NaN,NaN,"42,900원"
96,97,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",페인팅 발색 소프트 하게 아이라이너 젤펜슬 2개 아이라인펜슬,"할인 전 판매가37,500원",20%할인,"30,000원"
97,98,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",해외Maybelline 메이블린 라인 스틸레토 아이라이너 2개 Line Stilet...,NaN,NaN,"39,050원"
98,99,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",메이블린뉴욕 라인 타투 크레용 0.4g,NaN,NaN,"7,550원"
99,100,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",메이블린뉴욕 라인 타투 크레용 0.4g,NaN,NaN,"7,510원"


In [53]:
df.iloc[40:50]

,번호,이미지링크,제품제목,원래가격,할인률,현재가격
40,41,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",(3개)비러브 마이라인 펜 아이라이너 블랙,"할인 전 판매가11,400원",9%할인,"10,370원"
41,42,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",메소드메이크업 분장용 콤비펜슬 듀얼펜슬 블랙/브라운 아이브로우 아이라인,NaN,NaN,"5,200원"
42,43,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",애교살 아이라인 그리기 마스카라 도구,NaN,NaN,700원
43,44,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",비러브 마이라인 펜 아이라이너 블랙,"할인 전 판매가2,800원",9%할인,"2,540원"
44,45,https://shopping-phinf.pstatic.net/main_864141...,클리오 수퍼 프루프 펜라이너 0.55ml 아이라이너 아이라인 펜슬,NaN,NaN,"12,780원"
45,46,https://shopping-phinf.pstatic.net/main_862023...,아이라인그리기 눈화장가이드 애교살만들기 마스카라가드,"할인 전 판매가8,000원",60%할인,"3,200원"
46,47,https://shopping-phinf.pstatic.net/main_858610...,발색력좋은 펜슬 아이라인 번짐없는 토니모리 아이라인 롱브러쉬 브라운,"할인 전 판매가27,900원",46%할인,"14,900원"
47,48,https://shopping-phinf.pstatic.net/main_857237...,애교살 아이라인 가이드 실리콘 라이너 그리기 도구,"할인 전 판매가2,900원",10%할인,"2,610원"
48,49,https://shopping-phinf.pstatic.net/main_826788...,워터프루프 오앤에이 아이라이너 롱라스팅 브러쉬 초슬림 아이라인 펜슬 3색상,"할인 전 판매가16,000원",65%할인,"5,500원"
49,50,https://shopping-phinf.pstatic.net/main_844329...,부드럽게 원하는 라인을 연출 아이라이너 블랙,"할인 전 판매가38,190원",35%할인,"24,820원"


# 데이터 출력

# END